In [1]:
import matplotlib.pyplot as plt
import re  
import pprint
import numpy as np

In [2]:


def parse_latex_table(file_name):
    with open(file_name, "r") as f:
        content = f.read()

    # tables = content.split("\n\n")
    tables = re.split(r'\\end{tabular}', content)[:-1]

    data = {}
    for table in tables:
        lines = table.strip().split("\n")
        if len(lines) < 4:  
            continue
        ensemble = lines[2].split("&")[0].strip().replace('$_','_')
        data[ensemble] = {}

        for line in lines[3:-1]:  
            cols = line.split("&")
            person = cols[0].strip()
            results = [tuple(map(float, re.findall(r"([\d.]+)", item))) for item in cols[1:]]
            data[ensemble][person] = dict(zip(["N_E0", "Xi_EO", "Sigma_E0", "Lambda_E0", "Xi*_E0", "Delta_E0", "Sigma*_E0"], results))

    return data

file_name = "tables.txt"
data = parse_latex_table(file_name)
pprint.pprint(data)



FileNotFoundError: [Errno 2] No such file or directory: 'tables.txt'

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

def plot_ensemble(ensemble_data, ensemble_name,pdf):
    fig, ax = plt.subplots(figsize=(12, 8))

    offset = 0.05

    for i, (person, results) in enumerate(ensemble_data.items()):
        if person == '\\hline':
            continue
        valid_results = {k: v for k, v in results.items() if len(v) > 0}
        x = np.arange(len(valid_results))
        y = [result[0] for result in valid_results.values()]
        yerr = [result[1] *0.0001 if len(result) > 1 else 0 for result in valid_results.values()]
        ax.errorbar(x + i * offset, y, yerr=yerr, fmt="o", capsize=5, label=person)

    ax.set_xticks(x + (len(ensemble_data) - 1) * offset / 2)
    ax.set_xticklabels(valid_results.keys())
    ax.set_xlim(-0.5, len(valid_results) - 0.5 + (len(ensemble_data) - 1) * offset)

    ax.legend()
    ax.set_xlabel("Baryon")
    ax.set_ylabel("Fit Result")
    ax.set_title(f"Fit Results for {ensemble_name}")

    pdf.savefig(fig)
    plt.close(fig)

with PdfPages("output_plots.pdf") as pdf:
    for ensemble_name, ensemble_data in data.items():
        plot_ensemble(ensemble_data, ensemble_name, pdf)


